# *Cartesius* energy system model - version 1



In [43]:
# Importing libraries: ------
import pyomo.environ as pyo
import numpy as np
import pandas as pp

# Import data from excel: ---
def readInputFile(filename):
    EV_data = pp.read_excel(filename, sheet_name= 'EV_data', index_col=0)
    time = pp.read_excel(filename, sheet_name= 'time', index_col=0)
    #EV_chargers = pp.read_excel(filename, sheet_name= 'EV_chargers', index_col=0)
    grid_connection = pp.read_excel(filename, sheet_name= 'grid_connection', index_col=0)
    transformers = pp.read_excel(filename, sheet_name= 'transformers', index_col=0)
    HP = pp.read_excel(filename, sheet_name= 'HP', index_col=0)
    load = pp.read_excel(filename, sheet_name= 'load', index_col=0)
    BESS = pp.read_excel(filename, sheet_name= 'BESS', index_col=0)
    return {'EV_data':EV_data, 'time':time, 'grid_connection':grid_connection, 'transformers':transformers, 'HP':HP, 'load':load, 'BESS':BESS}
    # , 'EV_chargers':EV_chargers

filename = 'Initialized_Data.xlsx'
data = readInputFile(filename)
# ----------------------------
EV_data = data['EV_data']
time = data['time']
#EV_chargers = data['EV_chargers']
grid_connection = data['grid_connection']
transformers = data['transformers']
HP = data['HP']
load = data['load']
BESS = data['BESS']
# ----------------------------

# Create a model: ------------
model = pyo.ConcreteModel()


# Sets: ----------------------
model.T = pyo.Set(initialize=time.index, ordered=True, doc='Time')                          # Time [h1, ..., h24]
model.S = pyo.Set(initialize=EV_data.index, ordered=True, doc='Electric vehicles')          # EVs [EV1, ..., EVs]
#model.CH = pyo.Set(initialize=EV_chargers.index, ordered=True, doc='Chargers')              # Chargers [CH1, ..., CHch]
model.L = pyo.Set(initialize=load.index, ordered=True, doc='Loads')                         # Load [L1, ..., Ll]
model.B = pyo.Set(initialize=BESS.index, ordered=True, doc='BESS')                          # BESS [B1, ..., Bb]
model.H = pyo.Set(initialize=HP.index, ordered=True, doc='Heat pumps')                      # Heat Pumps [HP1, ..., HPh]
model.G = pyo.Set(initialize=grid_connection.index, ordered=True, doc= 'Grid connections')  # Grid Connection [POC1, ..., POCg]
model.TF = pyo.Set(initialize=transformers.index, ordered=True, doc='Transformers')         # Transformers [TRAFO1, ..., TRAFOtf]


# Electric vehicles:

Important parameters of the EVs are:
- Maximum charging power $P_{max}$ [kW]
- Maximum energy capacity $E_{max}$ [kWh]
- Arrival and departure time, $T_a$ and $T_d$ respectively
- Initial and final State of Charge (SOC) [%]

Subjected to the following constraints:
- $ 0 \leq P^{EV}_{s, t} \leq \overline{P^{EV}_{s}} \quad \forall t \; \forall s$

In [44]:
# ----------------------------
# EVs
# ----------------------------

# Parameters:
model.EV_Pmax = pyo.Param(model.S, initialize=EV_data['Max charging power [kW]'].to_dict(), mutable=True)       # Max power the s-th EV can be charged[kW]
model.EV_E = pyo.Param(model.S, initialize=EV_data['Energy capacity [kWh]'].to_dict(), mutable=True)            # Energy capacity of the s-th EV [kWh]
model.EV_Ta = pyo.Param(model.S, initialize=EV_data['Tarrival [h]'].to_dict(), mutable=True)                    # Arrival time of the s-th EV [h]*
model.EV_Td = pyo.Param(model.S, initialize=EV_data['Tdeparture [h]'].to_dict(), mutable=True)                  # Departure time of the s-th EV [h]*
model.EV_SOCa = pyo.Param(model.S, initialize=EV_data['Arrival SOC [%]'].to_dict(), mutable=True)               # s-th EV SOC at Ta [%]
model.EV_SOCd = pyo.Param(model.S, initialize=EV_data['Departure SOC [%]'].to_dict(), mutable=True)             # s-th EC SOC at Td [%]
model.EV_SOEa = pyo.Param(model.S, mutable=True)                                                                # s-th EV SOE at Ta [kWh]
model.EV_SOEd = pyo.Param(model.S, mutable=True)                                                                # s-th EC SOE at Td [kWh]

for s in model.S:
    model.EV_SOEa[s] = model.EV_E[s]*model.EV_SOCa[s]/100
    model.EV_SOEd[s] = model.EV_E[s]*model.EV_SOCd[s]/100


# Variables:
model.P_EV = pyo.Var(model.T, model.S, domain=pyo.NonNegativeReals)         # Power delivered to EV s-th at time t [kW]
#model.u_EV = pyo.Var(model.T, model.S, domain=pyo.Binary)                   # Binary variable of charging on/off of the s-th EV at time t


# Constraints:
model.EV_constraints = pyo.ConstraintList()

for s in model.S:
    for t in model.T:
        model.EV_constraints.add(expr=(model.P_EV[t,s] <= model.EV_Pmax[s]))                            # Constraints (...)
        #model.EV_constraints.add(expr=(model.P_EV[t,s] <= model.EV_Pmax[s] * model.u_EV[t,s]))          # Considering binary variables



# Heating system:

This section is for the electrical part of the heating system

Constraints:
- $ 0 \leq P^{HP}_{t, h} \leq \overline{P^{HP}_h} \quad \forall t \; \forall h $

In [45]:
# ----------------------------
# HPs 
# ----------------------------


# Parameters:
model.HP_Prated = pyo.Param(model.H, initialize=HP['Power rated [kW]'], mutable=True)       # rated power of the h-th heat pump [kW]


# Variables:
model.P_HP = pyo.Var(model.T, model.H, domain=pyo.NonNegativeReals)                 # Power delivered to HP h at time t [kW]


# Constraints:
model.HP_constraints = pyo.ConstraintList()

for h in model.H:
    for t in model.T:
        model.HP_constraints.add(expr=(model.P_HP[t,h] <= model.HP_Prated[h]))




# Grid connection:

Constraints:
- Power balance: $ \quad \sum_{s} P^{EV}_{t,s} + \sum_{h} P^{HP}_{t,h} + \sum_{g} P^{grid}_{t,g} = 0 \quad \forall t $

In [46]:
# Parameters: 
model.GC_Pmax = pyo.Param(model.G, initialize=grid_connection['Max power [kW]'], mutable=True)

# Variables:
model.P_grid = pyo.Var(model.T, model.G)                   # Power fed(+) or withdrewd(-) from the grid connection g at time t [kW]

# Constraints:

model.power_balance = pyo.ConstraintList()
model.GC_constraints = pyo.ConstraintList()
for t in model.T:
    model.power_balance.add(expr=(sum(model.P_EV[t,s] for s in model.S) + sum(model.P_HP[t,h] for h in model.H) + sum(model.P_grid[t,g] for g in model.G) == 0))
    for g in model.G:
        model.GC_constraints.add(expr=(model.P_grid[t,g] <= model.GC_Pmax[g]))
